# Quickstart guide to getting started querying the FANC dataset
This guide is adapted from https://github.com/htem/FANC_auto_recon/blob/main/example_notebooks/FANC_Connectomics_General_Intro.ipynb to be more wing-specific and less about 3d visualizations\
To explore the EM dataset, you need a caveclient token (see https://caveclient.readthedocs.io/en/latest/guide/authentication.html)\
To get verified as part of the FANC community see https://github.com/htem/FANC_auto_recon/wiki#collaborative-community\
You can explore just the 3d reconstructions of neurons without a token (see https://github.com/tuthill-lab/Lesser_Azevedo_2023/blob/main/plot_from_pkl.ipynb to get started)

In [42]:
## import packages
import pandas as pd
import numpy as np
from matplotlib import pyplot,patches
import matplotlib.pyplot as plt
import seaborn as sns
import os

from caveclient import CAVEclient # EM specific
from nglui.statebuilder import * # for making urls to neuroglancer view


In [2]:
## getting access to the dataset (after verification from Jasper, Leila, John, or Wei)

client = CAVEclient()
if not os.path.isfile(os.path.expanduser("~/.cloudvolume/secrets/cave-secret.json")):
    client.auth.get_new_token(open=True)

## if you have not yet setup this computer, uncomment this below line
## paste the token from the website in, and run the line

# client.auth.save_token(token="paste_token_here", overwrite=True)

## then comment or delete the line as you don't need to run it on this computer  again

datastack_name = 'fanc_production_mar2021'
client = CAVEclient(datastack_name)

In [3]:
## query annotation tables for cells of interest
wingmotor_df = client.materialize.query_table('wing_motor_neuron_table_v0')
neckmotor_df = client.materialize.query_table('neck_motor_neuron_table_v0')
halteremotor_df = client.materialize.query_table('haltere_motor_neuron_table_v0')

premotor_df = client.materialize.query_table('wing_premotor_v4') # biased toward preMNs for the left side MNs
sensory_df = client.materialize.query_table('nerve_bundle_fibers_v0') # this table is not very well maintained
connective_df = client.materialize.query_table('neck_connective') # DN labels not included

Table Owner Notice on soma_jan2022: Newer version of soma table is available. See client.info.get_datastack_info()['soma_table']
Table Owner Notice on soma_jan2022: Newer version of soma table is available. See client.info.get_datastack_info()['soma_table']


In [49]:
## add neurotransmitter info to premotor df
## From Lacin et. al., 2019
GABA_list = ['11B', '0A', '6A', '3B', '5B', '19A', '6B', '12B', '13B', '13A','9A','1B']
Glu_list = ['2A', '16B', '8A', '21A', '14A', '24B_'] #'15B', 
ACh_list = ['17A', '12A', '8B', '19B', '11A', '7B', '23B', '18B', '1A', '20A', '4B', '3A', '22A','10B','PSI']

premotor_df['sign'] = 'unknown'
premotor_df.loc[premotor_df['cell_type'].isin(GABA_list), 'sign'] = 'inh'
premotor_df.loc[premotor_df['cell_type'].isin(Glu_list), 'sign'] = 'inh'
premotor_df.loc[premotor_df['cell_type'].isin(ACh_list), 'sign'] = 'exc'
premotor_df.loc[premotor_df['classification_system'].isin(['sensory']), 'sign'] = 'exc'


In [4]:
##  to see info about table, like who created it and how complete it is

# tablename = 'neck_connective'
# client.materialize.get_table_metadata('peripheral_nerves')

In [11]:
## see the columns in a table
## most of the annotation tables have two columns for labels: "classification system" and "cell type", though these columns can mean different things depending on the table
## for the wing motor neurons table, classification system specifies the nerve and the side and cell type specifies the muscle innervated

wingmotor_df.head() 


,id,created,superceded_id,valid,classification_system,cell_type,pt_supervoxel_id,pt_root_id,pt_position
0,1,2022-11-30 17:07:21.586041+00:00,NaN,t,PDMN_L,DLM_1,72624531264718610,648518346486629017,"[17992, 120088, 2996]"
1,2,2022-11-30 17:07:21.586847+00:00,NaN,t,PDMN_L,DLM_2,72554162520753862,648518346515087943,"[14528, 119848, 3036]"
2,3,2022-11-30 17:07:21.587651+00:00,NaN,t,PDMN_L,DLM_3,72413355843006275,648518346486563887,"[11016, 118512, 2103]"
3,4,2022-11-30 17:07:21.588438+00:00,NaN,t,PDMN_L,DLM_4,72483793575108971,648518346494250631,"[13152, 120160, 2610]"
4,5,2022-11-30 17:07:21.589212+00:00,NaN,t,PDMN_L,DLM_5,73750361378386281,648518346472269618,"[49280, 117624, 915]"


In [10]:
print(set(wingmotor_df.classification_system))
print(set(wingmotor_df.cell_type))

{'ADMN_L', 'ADMN_R', 'mesoAN_R', 'PDMN_R', 'mesoAN_L', 'PDMN_L'}
{'ttb_u', 'DLM_3', 'PS2', 'PSn_u', 'hg4_u', 'nonMN_PSI', 'DVM_3b', 'ttc_u', 'tpn', 'DVM_1c', 'hg1_u', 'hg3', 'DVM_2b', 'DVM_1a', 'hg4', 'DLM_2', 'DVM_2a', 'iii1', 'DLM_4', 'nonMN_unk1', 'nonMN_unk3', 'dtpmn_u', 'DVM_1b', 'i2_u', 'tt', 'iii4a_u', 'PS1', 'hg3_u', 'DLM_5', 'DLM_1', 'iii3', 'iii3_dendrites', 'b2', 'tp2', 'hg2_u', 'iii4b_u', 'i1', 'b3_u', 'b1', 'nonMN_unk2', 'DVM_3a', 'hg2'}


In [ ]:
## Query synapses to see connectivity upstream or downstream of populations / single cells

In [20]:
## some possible neurons of interest
DLMs = wingmotor_df.loc[wingmotor_df.cell_type.str.startswith('DLM')].pt_root_id.to_list() # DLMs 1 - 5
giant_fiber = []
desc_ant = []

## 12B: 04728
ss04728 = premotor_df[premotor_df.cell_type.isin(['12B'])].pt_root_id.to_list()
## 12A: 48272
ss48272 = premotor_df[premotor_df.cell_type.isin(['12A'])][premotor_df.classification_system.isin(['local'])].pt_root_id.to_list() # remove primaries
## 3B/11B: 44002
ss44002 = [648518346477643278,648518346489463916,648518346488942057,648518346517121573,648518346494564151,648518346475387572,648518346483562133,648518346481006938,648518346470436606,648518346499869580,648518346489648435]

In [ ]:
query_neurons = DLMs

## post_ids to query upstream, pre_ids to query downstream
syn_df = client.materialize.synapse_query(post_ids = query_neurons)


In [19]:
## constrain synapse dataframe

subset = 'ss44002'

## pre_pt_root_id to constrain upstream, post_pt_root_id to constrain downstream
syn_df_filtered = syn_df[syn_df.pre_pt_root_id.isin(subset)]

In [28]:
# Another example: What preMNs are downstream of the GABAergic driverline and upstream of DLM MNs?

MNsyn_df = client.materialize.synapse_query(post_ids = query_neurons)
preMNsyn_df = client.materialize.synapse_query(pre_ids = subset)

syn_df_filtered = MNsyn_df[MNsyn_df.pre_pt_root_id.isin(MNsyn_df.post_pt_root_id.to_list())]


In [56]:
## see the top synaptic partners - constrained to neurons that are annotated

labels_df = pd.DataFrame(syn_df_filtered.pre_pt_root_id.value_counts().rename_axis('pt_root_id').reset_index(name='synapses')).merge(premotor_df[['pt_root_id','classification_system','cell_type','sign']],how = 'inner') 
labels_df[0:20]

,pt_root_id,synapses,classification_system,cell_type,sign
0,648518346517260837,1111,local,19B,exc
1,648518346494726315,1023,local,19B,exc
2,648518346524103685,249,local,6B,inh
3,648518346509851686,219,local,6B,inh
4,648518346499934975,203,local,5B,inh
5,648518346517261093,188,local,6B,inh
6,648518346486709217,166,local,7B,exc
7,648518346502325587,146,local_intersegmental,3B,inh
8,648518346480837344,145,local,6B,inh
9,648518346499909260,111,local,7B,exc


In [73]:
## link to a neuroglancer view

neuron_df = labels_df[0:50] # for rendering neurons of interest
syn_annotation_df = syn_df_filtered[syn_df_filtered.pre_pt_root_id.isin(neuron_df.pt_root_id.to_list())] # for putting point annotations at synapses, constrained to neurons rendered
color_dict = {'exc':'#02ab2e', 'inh':'#be0119', 'unknown':'#d8dcd6'} # green = excitatory, red = inhibitory, grey = unknown; to color the neurons and synapse points
neuron_df['color'] = neuron_df['sign'].map(color_dict).astype(str)

## specify data sources - do not change - 
# img_source = client.info.image_source()
img_source = 'precomputed://gs://neuroglancer/pinky100_v0/son_of_alignment_v15_rechunked'
seg_source = client.info.segmentation_source()
img_layer = ImageLayerConfig(name='fanc_v4',
                             source=img_source,
                             )
##

seg_layer = SegmentationLayerConfig(name = 'seg',
                                    source = seg_source,
                                    fixed_ids = neuron_df.pt_root_id.to_list())#,
                                    # color_column = 'color')

points = PointMapper(point_column='post_pt_position', set_position =True) ######################## change this to toggle rendering of pre- or post- synaptic points

anno_layer = AnnotationLayerConfig(name='postsynaptic sites')


sb = StateBuilder(layers=[img_layer, seg_layer, anno_layer], resolution=[4.3,4.3,45])
sb.render_state(return_as = 'html')




/var/folders/kc/3ksx1s8x04xcd7r2v8zsqqtm0000gn/T/ipykernel_32699/2790583215.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  neuron_df['color'] = neuron_df['sign'].map(color_dict).astype(str)
